In [1]:
import os
from dotenv import load_dotenv

import openai
import langchain
from langchain.llms import OpenAI

load_dotenv()

True

In [4]:
my_key = os.getenv('OPENAI_API_KEY')
# my_key

In [5]:
llm = OpenAI(temperature=0, openai_api_key=my_key)
text = 'What is AI?'
print(llm(text))

/Users/huke/openai-learning/venv2405/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/Users/huke/openai-learning/venv2405/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




AI stands for artificial intelligence. It is a branch of computer science that focuses on creating intelligent machines that can think and act like humans. AI involves the development of algorithms and computer programs that can learn from data, make decisions, and solve problems without explicit instructions from a human. It encompasses a wide range of technologies, including machine learning, natural language processing, and robotics. AI has applications in various industries, such as healthcare, finance, transportation, and entertainment. Its goal is to create machines that can perform tasks that typically require human intelligence, such as recognizing patterns, understanding language, and making decisions.
